In [1]:
import numpy as np
import os
from sklearn.decomposition import PCA
import tensorflow.keras as keras
from sklearn.model_selection import StratifiedShuffleSplit

In [2]:
X_per_class = []
labels = []

current_label = 0

for filename in os.listdir('nn_encoded'):
    plants = np.load('nn_encoded/' + filename)
    plant_name = filename.split('.')[0]
    
    X_per_class += [plants.reshape([plants.shape[0], -1])]
    labels += [current_label for _ in range(plants.shape[0])]
    current_label += 1

X_full = np.concatenate(X_per_class)
y_full = np.array(labels)

In [3]:
pca = PCA()

In [4]:
pca.fit(X_full)

PCA()

In [5]:
#np.eye(current_label)[np.array([0,1,2])]
y_full = np.eye(current_label)[y_full]

In [6]:
X_pca = pca.transform(X_full)

In [28]:
model = keras.models.Sequential([
    keras.layers.Input(shape=X_pca.shape[1:]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(6, activation="softmax")
])

In [29]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="sgd",
    metrics=["accuracy"]
)

In [30]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

def get_train_and_valid(design_matrix, labels):
    """
    Split dataset into training and validation sets.
    """
    for train_idx, valid_idx in split.split(design_matrix, labels):
        X_train = design_matrix[train_idx].copy()
        X_valid = design_matrix[valid_idx].copy()
        y_train = labels[train_idx].copy()
        y_valid = labels[valid_idx].copy()

    return X_train, y_train, X_valid, y_valid

In [31]:
X_train, y_train, X_valid, y_valid = get_train_and_valid(X_pca, y_full)

In [32]:
X_pca.shape[1:]

(180,)

In [34]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid))

Epoch 1/100
5/5 [==============================] - 0s 10ms/step - loss: 0.2011 - accuracy: 1.0000 - val_loss: 1.9930 - val_accuracy: 0.3056
Epoch 2/100
5/5 [==============================] - 0s 4ms/step - loss: 0.1803 - accuracy: 1.0000 - val_loss: 1.9964 - val_accuracy: 0.3056
Epoch 3/100
5/5 [==============================] - 0s 5ms/step - loss: 0.1846 - accuracy: 1.0000 - val_loss: 1.9981 - val_accuracy: 0.3056
Epoch 4/100
5/5 [==============================] - 0s 12ms/step - loss: 0.1708 - accuracy: 1.0000 - val_loss: 2.0016 - val_accuracy: 0.3056
Epoch 5/100
5/5 [==============================] - 0s 6ms/step - loss: 0.1597 - accuracy: 1.0000 - val_loss: 2.0036 - val_accuracy: 0.3056
Epoch 6/100
5/5 [==============================] - 0s 6ms/step - loss: 0.1664 - accuracy: 0.9931 - val_loss: 2.0069 - val_accuracy: 0.3056
Epoch 7/100
5/5 [==============================] - 0s 6ms/step - loss: 0.1449 - accuracy: 1.0000 - val_loss: 2.0109 - val_accuracy: 0.3056
Epoch 8/100
5/5 [========

In [ ]:
X_train.shape

In [ ]:
cumsum = np.cumsum(pca.explained_variance_ratio_)
for i, val in enumerate(pca.explained_variance_ratio_):
    print(i, "{:.4f} {:.4f}".format(100*val, 100*cumsum[i]))

In [ ]:
for i in range(180):
    print(X_pca[i, 0], y_full[i])